In [1]:
import pandas as pd


df = pd.read_csv("daily_data_crsp.csv", parse_dates=['date'])


FileNotFoundError: [Errno 2] No such file or directory: 'daily_data_crsp.csv'

In [ ]:
df.sort_values(by=['date', 'PERMNO'], inplace=True)
df

In [ ]:
df.columns = df.columns.str.lower()
df = df[(df['shrcd'].isin([10, 11])) & (df['exchcd'].isin([1, 2, 3]))]
daily_df = df
import gc
del df
gc.collect()


In [ ]:
daily_df.to_parquet('daily_df_crps_dump.parquet', compression='snappy')

In [ ]:
import pandas as pd
daily_df = pd.read_parquet('daily_df_crps_dump.parquet')
daily_df['date'] =  pd.to_datetime(daily_df['date'])
daily_df

In [ ]:
import wrds
import pandas as pd

conn = wrds.Connection()

market_df = conn.raw_sql("""
    SELECT date, vwretd
    FROM crsp.dsi
    WHERE date BETWEEN '1968-01-01' AND '2024-12-31'
    ORDER BY date
""", date_cols=['date'])

# Rename for clarity
market_df.rename(columns={'vwretd': 'mkt_ret'}, inplace=True)

In [ ]:
market_df

In [ ]:
daily_df = daily_df.join(market_df.set_index('date'), on='date', how='left')


In [ ]:
daily_df.reset_index(drop=True, inplace=True)
daily_df

In [ ]:
daily_df.to_parquet('daily_df_mkt_ret.parquet', compression='snappy')

In [ ]:
import pandas as pd
daily_df = pd.read_parquet('daily_df_mkt_ret.parquet')
daily_df['date'] =  pd.to_datetime(daily_df['date'])
daily_df

In [ ]:
daily_df.dropna(subset=['ret', 'mkt_ret'], inplace=True)
daily_df

In [ ]:
daily_df['month'] = daily_df['date'].dt.to_period('M').dt.to_timestamp('M')

In [ ]:
def compute_rolling_corr_vol(df, window_corr=1260, window_vol=252, progress_step=500):
    results = []
    stock_groups = df.groupby('permno')
    total_stocks = len(stock_groups)

    print(f"Computing rolling correlation and volatility for {total_stocks} stocks...")

    for i, (permno, stock_df) in enumerate(stock_groups):
        stock_df = stock_df.sort_values('date').copy()
        stock_df.set_index('date', inplace=True)
        stock_df['corr_5y'] = stock_df['ret'].rolling(window=window_corr, min_periods=200).corr(stock_df['mkt_ret'])
        stock_df['vol_1y'] = stock_df['ret'].rolling(window=window_vol, min_periods=100).std()

        stock_df = stock_df.reset_index()
        stock_df['month'] = stock_df['date'].dt.to_period('M').dt.to_timestamp('M')
        monthly_df = stock_df.groupby('month').last().reset_index()
        monthly_df['permno'] = permno

        results.append(monthly_df[['permno', 'month', 'corr_5y', 'vol_1y']])
        if (i + 1) % progress_step == 0 or (i + 1) == total_stocks:
            print(f"Processed {i + 1} / {total_stocks} stocks")

    return pd.concat(results, ignore_index=True)


In [ ]:
rolling_df = compute_rolling_corr_vol(daily_df)
